In [2]:
import numpy as np
import pandas as pd
import requests
import json
import psycopg2
from sqlalchemy import create_engine
import os
import pymongo
propublica_token = os.environ['propub_token']
postgres_password = os.environ['POSTGRES_PASSWORD']
mongo_username = os.environ['MONGO_INITDB_ROOT_USERNAME']
mongo_password = os.environ['MONGO_INITDB_ROOT_PASSWORD']
mongo_init_db = os.environ['MONGO_INITDB_DATABASE']
import getdata
members_vv, cvote_vv, memvotes_vv = getdata.get_voteview()
useragent = getdata.get_useragent()
members_pp = getdata.get_propublica(propublica_token, useragent, email='jkropko@virginia.edu')
members = getdata.merge_members(members_pp=members_pp, members_vv=members_vv)
members.head(1).T
members['propublica_id'].isna().value_counts()
cvote_vv.head(1).T
memvotes_vv.head(1).T
server = psycopg2.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432')
server.autocommit = True
cursor = server.cursor()
try:
    cursor.execute("CREATE DATABASE contrans")
except:
    cursor.execute("DROP DATABASE contrans")
    cursor.execute("CREATE DATABASE contrans")

engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432',
    db = 'contrans'))
members.to_sql('members', con=engine, if_exists='replace', index=False, chunksize=1000)
memvotes_vv.to_sql('member_vote', con=engine, if_exists='replace', index=False, chunksize=1000)
cvote_vv.to_sql('rollcalls', con=engine, if_exists='replace', index=False, chunksize=1000)

myquery = '''
SELECT * FROM members
'''
rollcall = pd.read_sql(myquery, con=engine)
rollcall.head(1).T

bills, num_results = getdata.get_bills_pp(propublica_token, useragent, email='jkropko@virginia.edu')